In [1]:
import os

In [2]:
%pwd

'C:\\Users\\suraj\\Kidney-Disease-classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'C:\\Users\\suraj\\Kidney-Disease-classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from src.CnnClassifier.constants import *
from src.CnnClassifier.utils.common import read_yaml,create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [11]:
import zipfile
import gdown
from src.CnnClassifier import logger
from src.CnnClassifier.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-10 14:16:07,818: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-10 14:16:07,829: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-10 14:16:07,829: INFO: common: created directory at: artifacts]
[2025-06-10 14:16:07,835: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-10 14:16:07,835: INFO: 2140807067: Downloading data from https://drive.google.com/file/d/1sTXnAViiu4j7dxh3xKb4xAA_vmQasNnL/view?usp=sharing into file artifacts/data_ingestion/dataset.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1sTXnAViiu4j7dxh3xKb4xAA_vmQasNnL
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1sTXnAViiu4j7dxh3xKb4xAA_vmQasNnL&confirm=t&uuid=8dc275d7-b974-42c0-a84f-d1e09184d418
To: C:\Users\suraj\Kidney-Disease-classification\artifacts\data_ingestion\dataset.zip
100%|██████████| 940M/940M [00:30<00:00, 30.4MB/s] 

[2025-06-10 14:16:42,561: INFO: 2140807067: Downloaded data from https://drive.google.com/file/d/1sTXnAViiu4j7dxh3xKb4xAA_vmQasNnL/view?usp=sharing into file artifacts/data_ingestion/dataset.zip]
